In [ ]:
!nvidia-smi

In [ ]:
!pip install huggingface_hub
!pip install diffusers==0.2.4
!pip install transformers scipy ftfy
!pip install "ipywidgets>=7,<8"
!nvidia-smi
!pip install git+https://github.com/openai/CLIP.git 
!pip install realesrgan

In [ ]:
1+1

In [ ]:
!nvidia-smi

In [ ]:
import torch
from torch import autocast, nn
from PIL import Image
import random
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, PNDMScheduler
from diffusers import LMSDiscreteScheduler
from torch import autocast
import argparse
import cv2
import glob
import os
from basicsr.archs.rrdbnet_arch import RRDBNet
from realesrgan import RealESRGANer
from realesrgan.archs.srvgg_arch import SRVGGNetCompact

import clip
import os
import numpy as np
import torch.nn.functional as nnf
import sys
from typing import Tuple, List, Union, Optional
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import skimage.io as io

import transformers


torch_device = "cuda:3" if torch.cuda.is_available() else "cpu"


# 1. Load the autoencoder model which will be used to decode the latents into image space. 
vae = AutoencoderKL.from_pretrained("../models/spaceforge-diffusion/vae")
# 2. Load the tokenizer and text encoder to tokenize and encode the text. 
tokenizer = CLIPTokenizer.from_pretrained("../models/clip-vit-large-patch14/")
text_encoder = CLIPTextModel.from_pretrained("../models/clip-vit-large-patch14/")

# 3. The UNet model for generating the latents.
unet = UNet2DConditionModel.from_pretrained("../models/spaceforge-diffusion/unet/")

vae = vae.to(torch_device)
text_encoder = text_encoder.to(torch_device)
unet = unet.to(torch_device)


SRmodel = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
netscale = 4
model_path='../models/gans/RealESRGAN_x4plus.pth'
upsampler = RealESRGANer(
        scale=netscale,
        model_path=model_path,
        model=SRmodel,
        tile=0,
        tile_pad=10,
        pre_pad=0,
        half=True)

In [ ]:
prompt=['an oil painting of a rabbit walking in the desert']

height = 640                        # default height
width = 512                         # default width 

num_inference_steps = 50            # Number of denoising steps
guidance_scale = 12.5                 # Scale for classifier-free guidance

generator = torch.manual_seed(random.randint(0,250000000000))   # Seed generator to create the inital latent noise

scheduler = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", num_train_timesteps=999)

batch_size = 1
text_input = tokenizer(prompt, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")

with torch.no_grad():
    text_embeddings = text_encoder(text_input.input_ids.to(torch_device))[0]

max_length = text_input.input_ids.shape[-1]
uncond_input = tokenizer(
    [""] * batch_size, padding="max_length", max_length=max_length, return_tensors="pt"
)


with torch.no_grad():
    uncond_embeddings = text_encoder(uncond_input.input_ids.to(torch_device))[0]

text_embeddings = torch.cat([uncond_embeddings, text_embeddings])

latents = torch.randn(
  (batch_size, unet.in_channels, height // 8, width // 8),
  generator=generator,
)
random_latents=latents
latents = latents.to(torch_device)

scheduler.set_timesteps(num_inference_steps)
latents = latents * scheduler.sigmas[0]

with autocast("cuda"):
  
  for i, t in tqdm(enumerate(scheduler.timesteps)):
    # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
    latent_model_input = torch.cat([latents] * 2)
    sigma = scheduler.sigmas[i]
    latent_model_input = latent_model_input / ((sigma**2 + 1) ** 0.5)

    # predict the noise residual
    with torch.no_grad():
        noise_pred = unet(latent_model_input, t, encoder_hidden_states=text_embeddings)["sample"]

    # perform guidance
    noise_pred_uncond ,noise_pred_text = noise_pred.chunk(2)
    noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)
    # compute the previous noisy sample x_t -> x_t-1
    latents = scheduler.step(noise_pred, i, latents)["prev_sample"]

# scale and decode the image latents with vae
latents = 1 / 0.18215 * latents

with torch.no_grad():
  image = vae.decode(latents)

image = (image / 2 + 0.5).clamp(0, 1)
img=image.detach().cpu().permute(0, 2,3, 1).squeeze(0).numpy()*255
image = image.detach().cpu().permute(0, 2, 3, 1).numpy()

images = (image * 255).round().astype("uint8")
pil_images = [Image.fromarray(image) for image in images]
pil_images[0]

In [ ]:
vx=1.0+(0.00)
vy=.00
rl=torch.exp((random_latents*vx))+vy
rl=torch.log(rl)
std,mean=torch.std_mean(rl)
rl=(rl-mean)/std

In [ ]:
rl

In [ ]:
prompt=['an oil painting of a lion walking in the desert']
height = 640                        # default height
width = 512                         # default width 

num_inference_steps = 50            # Number of denoising steps
guidance_scale = 10                 # Scale for classifier-free guidance

generator = torch.manual_seed(777)#random.randint(0,25000000))   # Seed generator to create the inital latent noise

scheduler = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", num_train_timesteps=1000)

batch_size = 1
text_input = tokenizer(prompt, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")

with torch.no_grad():
    text_embeddings = text_encoder(text_input.input_ids.to(torch_device))[0]

max_length = text_input.input_ids.shape[-1]
uncond_input = tokenizer(
    [""] * batch_size, padding="max_length", max_length=max_length, return_tensors="pt"
)
with torch.no_grad():
    uncond_embeddings = text_encoder(uncond_input.input_ids.to(torch_device))[0]

text_embeddings = torch.cat([uncond_embeddings, text_embeddings])

latents = torch.randn(
  (batch_size, unet.in_channels, height // 8, width // 8),
  generator=generator,
)
latents=rl
latents = latents.to(torch_device)

scheduler.set_timesteps(num_inference_steps)
latents = latents * scheduler.sigmas[0]

with autocast("cuda"):
  
  for i, t in tqdm(enumerate(scheduler.timesteps)):
    # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
    latent_model_input = torch.cat([latents] * 2)
    sigma = scheduler.sigmas[i]
    latent_model_input = latent_model_input / ((sigma**2 + 1) ** 0.5)

    # predict the noise residual
    with torch.no_grad():
        noise_pred = unet(latent_model_input, t, encoder_hidden_states=text_embeddings)["sample"]

    # perform guidance
    noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
    noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)
    # compute the previous noisy sample x_t -> x_t-1
    latents = scheduler.step(noise_pred, i, latents)["prev_sample"]

# scale and decode the image latents with vae
latents = 1 / 0.18215 * latents

with torch.no_grad():
    image = vae.decode(latents)

image = (image / 2 + 0.5).clamp(0, 1)
img=image.detach().cpu().permute(0, 2,3, 1).squeeze(0).numpy()*255
image = image.detach().cpu().permute(0, 2, 3, 1).numpy()

images = (image * 255).round().astype("uint8")
pil_images = [Image.fromarray(image) for image in images]
pil_images[0]

In [ ]:
output, _ = upsampler.enhance(img, outscale=4)
out_image = Image.fromarray(output)


In [ ]:
out_image

In [1]:
from google_trans_new import google_translator  
  
translator = google_translator()  


In [11]:
translate_text = translator.translate("a great person", lang_tgt='en')  


In [12]:
translate_text

'a great person '